In [224]:
# import libraries
import requests
import re
import pyjq
import json
from newspaper import Article
import pymongo
import datetime
from dateutil.rrule import rrule, MONTHLY
import math
import time

client = pymongo.MongoClient (host="da1.eecs.utk.edu")
db = client ['GDA']
coll = db ['NYT']
#clean this up for now
coll.drop()

start = 1960
end = 2020
years = []
for i in range(end-start):
    years.append(start + i)

api_key = 'mXe9rrak6WGZA7H2zzeGejAbh0LoCAzY'

for i in range(len(years)):    
    url_out = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&fq=pub_year:("'+str(years[i])+'")&api-key=' + api_key
    r = requests .get(url_out)
    json_data = r.json()
    meta = pyjq.all('.response .meta',json_data)
    pages = math.ceil(meta[0]['hits']/10)
    print('Working on Year {0}. There are {1} page(s)'.format(years[i],pages))
    for p in range(pages):   
        url_in = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=student+loan&fq=pub_year:("'+str(years[i])+'")&page='+str(p)+'&api-key='+ api_key
        #print('Page {0} of {1}'.format(p,pages))
        r = requests .get(url_in)
        json_data = r.json()
        count = pyjq.all('.response .docs | length',json_data)[0]
        urls = pyjq.all('.response .docs[] .web_url',json_data)

        for j in range(count):
            toi_article = Article(urls[j], language="en")
            try:
                toi_article.download() 
            except:
                continue
            try:
                toi_article.parse() 
            except:
                continue
            try:
                toi_article.nlp() 
            except:
                continue
            coll.insert_one ( {'title':toi_article.title,
                   'text': toi_article.text,
                   'summary': toi_article.summary, 
                   'keywords': toi_article.keywords, 
                   'year':years[i],
                   'url': urls[j] } )
        time.sleep(6)

Working on Year 1960. There are 4 page(s)
Working on Year 1961. There are 3 page(s)
Working on Year 1962. There are 2 page(s)
Working on Year 1963. There are 2 page(s)
Working on Year 1964. There are 56 page(s)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-224-c2930ae6e44a>", line 63, in <module>
    time.sleep(6)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 313, in wrappe

KeyboardInterrupt: 